In [1]:
from z3 import Solver
from meyer.meyer import conclude
from meyer.program import prog, progs
from meyer.equivalence import eq_pre
from meyer.special_programs import Skip
from meyer.util.z3py_set import set, sets

s = Solver()

In [2]:
title = "P26 (p \ C) = (p; (C: Skip))"
p = prog(s, 'p')
C = set('C')
conclude(s, p // C == p ^ (Skip() / C), title)

P26 (p \ C) = (p; (C: Skip))
Universe = U, has 3 element(s)
Holds: unsat 



In [7]:
title = "P27 (p1 ∪ p2) \ C = (p1 \ C) ∪ (p2 \ C)"
p1, p2 = progs(s, 'p1 p2')
C = set('C')
s.add(eq_pre(p1, p2)) # Additional assumption but too strong
conclude(s, (p1 | p2) // C == (p1 // C) | (p2 // C), title)

P27 (p1 ∪ p2) \ C = (p1 \ C) ∪ (p2 \ C)
Universe = U, has 3 element(s)
Holds: unsat 



In [8]:
title = "P28 (p1 ; p2) \ C = p1 ; (p2 \ C)"
p1, p2 = progs(s, 'p1 p2')
C = set('C')
conclude(s, (p1 ^ p2) // C == p1 ^ (p2 // C), title)

P28 (p1 ; p2) \ C = p1 ; (p2 \ C)
Universe = U, has 3 element(s)
Holds: unsat 



Ha　Corest, 型がおかしい，p.post，特殊な制約

In [15]:
title = "P29 (p \ C) ⊆ p but (p \ C) ⊆ C"
p = prog(s, 'p')
C = set('C')
s.add(p.pre <= (p.post() << C)) # Additional assumption but too naive
conclude(s, p // C <= p, title)

P29 (p \ C) ⊆ p but (p \ C) ⊆ C
Universe = U, has 3 element(s)
Holds: unsat 



In [30]:
title = "P30 If D ⊆ C then (p \ D) ⊆ (p \ C)"
p = prog(s, 'p')
D, C = sets('D C')
s.add(p.pre() <= (p.post() << C) & (p.post() << D)) # Additional assumption but too naive
s.add(D <= C)
conclude(s, p // D <= p // C, title)

P30 If D ⊆ C then (p \ D) ⊆ (p \ C)
Universe = U, has 3 element(s)
Holds: unsat 

